In [1]:
import os
import click
import pandas as pd
import numpy as np

In [2]:
def smartAppend(table,name,value):
    """ helper function for appending in a dictionary """
    if name not in table.keys():
        table[name] = []
    table[name].append(value)

In [3]:
# @click.command()
# @click.option('--file-with-filenames', required=True)
# @click.option(
#     "--output-folder", required=False, default=""
# )  # by default current directory, where you are running your script from

In [4]:
output_folder = ""

In [5]:
# with open(file_with_filenames, encoding='utf-8') as f:
#     list_of_files = f.readlines()

In [6]:
list_of_files = ['/share/ScratchGeneral/anncuo/OneK1K/CRM_interaction/Bcells_Bcell_eQTLs/ABRACL.tsv',
                  '/share/ScratchGeneral/anncuo/OneK1K/CRM_interaction/Bcells_Bcell_eQTLs/AC006129.4.tsv']

In [7]:
x = 0
table = {}
for file in list_of_files:
#     print(file)
    x += 1
    if x % 500 == 0: 
        print(x)
    df = pd.read_csv(file, index_col=0)
#     print(df.head())
    nsnps = int(len(df))
    if nsnps==0:
        continue
    gene = os.path.splitext(os.path.basename(file))[0] 
#     print(gene)
    chrom = df['chrom'].values[0]
#     print(chrom)
    for i in range(nsnps):
        temp = {}
        temp['gene'] = gene
        temp['n_snps'] = nsnps
        temp['snp_id'] = df['variant'].values[i]
        temp['pv_raw'] = df['pv'].values[i]
        temp['pv_Bonf'] = nsnps * temp['pv_raw']
        if temp['pv_Bonf']>1: temp['pv_Bonf'] = 1
        if temp['pv_Bonf']<0: temp['pv_Bonf'] = 0

    for key in temp.keys():
        smartAppend(table, key, temp[key])

# print(x)
for key in table.keys():
    table[key] = np.array(table[key])

df = pd.DataFrame.from_dict(table)
print(df.head())
outfile = "summary.csv" 
myp = os.path.join(output_folder, outfile)
# myp
#     df.to_csv(myp)

         gene  n_snps       snp_id    pv_raw   pv_Bonf
0      ABRACL       1  6:139343892  0.005028  0.005028
1  AC006129.4       1  19:42049821  0.020503  0.020503


In [17]:
# pip install qvalue

# import qvalue as qvalue

# ?qvalue.estimate

# df_sign = df
# df_sign["qv"] = qvalue.estimate(df_sign["pv_Bonf"])

In [20]:
import scipy as sp
from scipy import interpolate


def qv_estimate(pv, m=None, verbose=False, lowmem=False, pi0=None):
    """
    Estimates q-values from p-values
    Args
    =====
    m: number of tests. If not specified m = pv.size
    verbose: print verbose messages? (default False)
    lowmem: use memory-efficient in-place algorithm
    pi0: if None, it's estimated as suggested in Storey and Tibshirani, 2003.
         For most GWAS this is not necessary, since pi0 is extremely likely to be
         1
    """
    assert(pv.min() >= 0 and pv.max() <= 1), "p-values should be between 0 and 1"

    original_shape = pv.shape
    pv = pv.ravel()  # flattens the array in place, more efficient than flatten()

    if m is None:
        m = float(len(pv))
    else:
        # the user has supplied an m
        m *= 1.0

    # if the number of hypotheses is small, just set pi0 to 1
    if len(pv) < 100 and pi0 is None:
        pi0 = 1.0
    elif pi0 is not None:
        pi0 = pi0
    else:
        # evaluate pi0 for different lambdas
        pi0 = []
        lam = sp.arange(0, 0.90, 0.01)
        counts = sp.array([(pv > i).sum() for i in sp.arange(0, 0.9, 0.01)])
        for l in range(len(lam)):
            pi0.append(counts[l]/(m*(1-lam[l])))

        pi0 = sp.array(pi0)

        # fit natural cubic spline
        tck = interpolate.splrep(lam, pi0, k=3)
        pi0 = interpolate.splev(lam[-1], tck)
        if verbose:
            print("qvalues pi0=%.3f, estimated proportion of null features " % pi0)

        if pi0 > 1:
            if verbose:
                print("got pi0 > 1 (%.3f) while estimating qvalues, setting it to 1" % pi0)
            pi0 = 1.0

    assert(pi0 >= 0 and pi0 <= 1), "pi0 is not between 0 and 1: %f" % pi0

    if lowmem:
        # low memory version, only uses 1 pv and 1 qv matrices
        qv = sp.zeros((len(pv),))
        last_pv = pv.argmax()
        qv[last_pv] = (pi0*pv[last_pv]*m)/float(m)
        pv[last_pv] = -sp.inf
        prev_qv = last_pv
        for i in range(int(len(pv))-2, -1, -1):
            cur_max = pv.argmax()
            qv_i = (pi0*m*pv[cur_max]/float(i+1))
            pv[cur_max] = -sp.inf
            qv_i1 = prev_qv
            qv[cur_max] = min(qv_i, qv_i1)
            prev_qv = qv[cur_max]

    else:
        p_ordered = sp.argsort(pv)
        pv = pv[p_ordered]
        qv = pi0 * m/len(pv) * pv
        qv[-1] = min(qv[-1], 1.0)

        for i in range(len(pv)-2, -1, -1):
            qv[i] = min(pi0*m*pv[i]/(i+1.0), qv[i+1])

        # reorder qvalues
        qv_temp = qv.copy()
        qv = sp.zeros_like(qv)
        qv[p_ordered] = qv_temp

    # reshape qvalues
    qv = qv.reshape(original_shape)

    return qv

In [26]:
df["qv"] = qv_estimate(df["pv_Bonf"])

/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:72: DeprecationWarning: scipy.argsort is deprecated and will be removed in SciPy 2.0.0, use numpy.argsort instead
/share/ScratchGeneral/anncuo/jupyter/conda_notebooks/envs/cellregmap_notebook/lib/python3.7/site-packages/ipykernel_launcher.py:82: DeprecationWarning: scipy.zeros_like is deprecated and will be removed in SciPy 2.0.0, use numpy.zeros_like instead


In [39]:
fdr=0.02

In [40]:
df_sign = df[df["qv"]<fdr]

In [41]:
df.shape

(2, 6)

In [42]:
df_sign.shape

(1, 6)

In [43]:
df_sign.head()

,gene,n_snps,snp_id,pv_raw,pv_Bonf,qv
0,ABRACL,1,6:139343892,0.005028,0.005028,0.010055


In [ ]:
##### 

In [39]:
pip install rpy2

     |████████████████████████████████| 202 kB 17.7 MB/s eta 0:00:01
     |████████████████████████████████| 70 kB 48.1 MB/s eta 0:00:01
     |████████████████████████████████| 339 kB 59.3 MB/s eta 0:00:01
  Created wheel for rpy2: filename=rpy2-3.5.2-cp37-cp37m-linux_x86_64.whl size=368165 sha256=099ba8df4a34896c87f2a538ec0c37f176ea55f228506fa3fb9ef32807d6cbf3
  Stored in directory: /tmp/pip-ephem-wheel-cache-5_sod446/wheels/b8/27/91/fafb1fc786bd06e26f89321039aff9223d1e16acfbdb2572fd
Successfully built rpy2
Note: you may need to restart the kernel to use updated packages.


In [43]:
import sys
import rpy2
import rpy2.robjects as ro
from rpy2.robjects import r
from rpy2.robjects.packages import importr

devtools = importr("devtools")
qvalue = importr("qvalue")

PackageNotInstalledError: The R package "qvalue" is not installed.